In [2]:
pip freeze > model_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [19]:
data_mm_train_df = pd.read_csv('data_smin_mm_train_csv')
data_mm_test_df = pd.read_csv('data_mm_test_csv')
data_mm_val_df = pd.read_csv('data_mm_val_csv')

In [20]:
data_ss_train_df = pd.read_csv('data_smin_ss_train_csv')
data_ss_test_df = pd.read_csv('data_ss_test_csv')
data_ss_val_df = pd.read_csv('data_ss_val_csv')

In [21]:
data_log_train_df = pd.read_csv('data_smin_log_train_csv')
data_log_test_df = pd.read_csv('data_test_csv')
data_log_val_df = pd.read_csv('data_val_csv')

In [5]:
print(f"Data_train: {data_mm_train_df.shape}")
print(f"Data_test: {data_mm_test_df.shape}")
print(f"Data_val: {data_mm_val_df.shape}")

#Divide by 10 for the sample

Data_train: (288041, 10)
Data_test: (27022, 10)
Data_val: (54045, 10)


In [6]:
def data_format2(train_df, test_df, val_df, sample):
    train_df = train_df.drop(['Unnamed: 0'], axis=1)
    test_df = test_df.drop(['Unnamed: 0'], axis=1)
    val_df = val_df.drop(['Unnamed: 0'], axis=1)

    if sample!=None:
        train_df = train_df.sample(n=14402, replace=False, random_state=7, axis=0)
        test_df = test_df.sample(n=1351, replace=False, random_state=7, axis=0)
        val_df = val_df.sample(n=2702, replace=False, random_state=7, axis=0)

    X_train = train_df.drop('Winner_num',axis=1)
    y_train = train_df['Winner_num']

    X_test= test_df.drop('Winner_num',axis=1) 
    y_test = test_df['Winner_num']

    X_val= val_df.drop('Winner_num',axis=1) 
    y_val = val_df['Winner_num']
    return(X_train, X_test, X_val, y_train, y_test, y_val)

In [7]:
def data_format(train_df, test_df, val_df, sample):
    train_df = train_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
    test_df = test_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)
    val_df = val_df.drop(['Unnamed: 0', 'Remaining overs and balls', 'Remaining Team Value', 'Remaining Average Team Value'], axis=1)

    if sample!=None:
        train_df = train_df.sample(n=14402, replace=False, random_state=7, axis=0)
        test_df = test_df.sample(n=1351, replace=False, random_state=7, axis=0)
        val_df = val_df.sample(n=2702, replace=False, random_state=7, axis=0)

    X_train = train_df.drop('Winner_num',axis=1)
    y_train = train_df['Winner_num']

    X_test= test_df.drop('Winner_num',axis=1) 
    y_test = test_df['Winner_num']

    X_val= val_df.drop('Winner_num',axis=1) 
    y_val = val_df['Winner_num']
    return(X_train, X_test, X_val, y_train, y_test, y_val)

In [7]:
X_mm_train_sample, X_mm_test_sample, X_mm_val_sample, y_mm_train_sample, y_mm_test_sample, y_mm_val_sample = data_format2(data_mm_train_df, data_mm_test_df, data_mm_val_df, 'sample')
X_ss_train_sample, X_ss_test_sample, X_ss_val_sample, y_ss_train_sample, y_ss_test_sample, y_ss_val_sample = data_format2(data_ss_train_df, data_ss_test_df, data_ss_val_df, 'sample')
X_log_train_sample, X_log_test_sample, X_log_val_sample, y_log_train_sample, y_log_test_sample, y_log_val_sample = data_format2(data_log_train_df, data_log_test_df, data_log_val_df, 'sample')

In [22]:
X_ss_train, X_ss_test, X_ss_val, y_ss_train, y_ss_test, y_ss_val = data_format2(data_ss_train_df, data_ss_test_df, data_ss_val_df, None)

In [8]:
X_mm_train_sample, X_mm_test_sample, X_mm_val_sample, y_mm_train_sample, y_mm_test_sample, y_mm_val_sample = data_format(data_mm_train_df, data_mm_test_df, data_mm_val_df, 'sample')
X_ss_train_sample, X_ss_test_sample, X_ss_val_sample, y_ss_train_sample, y_ss_test_sample, y_ss_val_sample = data_format(data_ss_train_df, data_ss_test_df, data_ss_val_df, 'sample')
X_log_train_sample, X_log_test_sample, X_log_val_sample, y_log_train_sample, y_log_test_sample, y_log_val_sample = data_format(data_log_train_df, data_log_test_df, data_log_val_df, 'sample')

In [9]:
#Put in here the code to get out the X_trains etc

SVM model

In [10]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [11]:
adfasdf

NameError: name 'adfasdf' is not defined

Baseline one SVC

In [38]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = SVC(kernel="linear", random_state=7)

#Train the model using the training sets
clf.fit(X_ss_train_sample, y_ss_train_sample)

#Predict the response for test dataset
y_pred = clf.predict(X_ss_test_sample)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_ss_test_sample, y_pred))

In [39]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_ss_test_sample, y_pred))

Accuracy: 0.6039970392301999


In [ ]:
sdgdfg

NameError: name 'sdgdfg' is not defined

In [ ]:
k_list = ['linear', 'rbf', 'sigmoid']
gamma_list = [0.001, 0.01, 0.1, 1, 10, 100,200, 300]
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 200, 300]

In [ ]:
print("MinMax")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_mm_train_sample, y_mm_train_sample)
            y_mm_pred = model.predict(X_mm_val_sample)
            accuracy = accuracy_score(y_mm_val_sample, y_mm_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

MinMax
linear, Gamma: 0.001, C: 0.001, Accuracy: 0.009622501850481125
linear, Gamma: 0.001, C: 0.01, Accuracy: 0.5851221317542561
linear, Gamma: 0.001, C: 0.1, Accuracy: 0.6039970392301999
linear, Gamma: 0.001, C: 1, Accuracy: 0.5943745373797187
linear, Gamma: 0.001, C: 10, Accuracy: 0.5932642487046632
linear, Gamma: 0.001, C: 100, Accuracy: 0.5932642487046632
linear, Gamma: 0.001, C: 200, Accuracy: 0.5932642487046632
linear, Gamma: 0.001, C: 300, Accuracy: 0.5928941524796447
linear, Gamma: 0.01, C: 0.001, Accuracy: 0.009622501850481125
linear, Gamma: 0.01, C: 0.01, Accuracy: 0.5851221317542561
linear, Gamma: 0.01, C: 0.1, Accuracy: 0.6039970392301999
linear, Gamma: 0.01, C: 1, Accuracy: 0.5943745373797187
linear, Gamma: 0.01, C: 10, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 100, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 200, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 300, Accuracy: 0.5928941524796447
linear, Gamma: 0.1, C: 0.001, Accuracy: 0.00962250185

In [42]:
print("SS")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_ss_train_sample, y_ss_train_sample)
            y_ss_pred = model.predict(X_ss_val_sample)
            accuracy = accuracy_score(y_ss_val_sample, y_ss_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

SS
linear, Gamma: 0.001, C: 0.001, Accuracy: 0.5858623242042931
linear, Gamma: 0.001, C: 0.01, Accuracy: 0.5903034789045152
linear, Gamma: 0.001, C: 0.1, Accuracy: 0.5921539600296077
linear, Gamma: 0.001, C: 1, Accuracy: 0.5932642487046632
linear, Gamma: 0.001, C: 10, Accuracy: 0.5932642487046632
linear, Gamma: 0.001, C: 100, Accuracy: 0.5928941524796447
linear, Gamma: 0.001, C: 200, Accuracy: 0.5932642487046632
linear, Gamma: 0.001, C: 300, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 0.001, Accuracy: 0.5858623242042931
linear, Gamma: 0.01, C: 0.01, Accuracy: 0.5903034789045152
linear, Gamma: 0.01, C: 0.1, Accuracy: 0.5921539600296077
linear, Gamma: 0.01, C: 1, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 10, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 100, Accuracy: 0.5928941524796447
linear, Gamma: 0.01, C: 200, Accuracy: 0.5932642487046632
linear, Gamma: 0.01, C: 300, Accuracy: 0.5932642487046632
linear, Gamma: 0.1, C: 0.001, Accuracy: 0.5858623242042931
li

In [ ]:
kgkjjk

In [43]:
print("LogNorm")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_log_train_sample, y_log_train_sample)
            y_log_pred = model.predict(X_log_val_sample)
            accuracy = accuracy_score(y_log_val_sample, y_log_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

LogNorm
linear, Gamma: 0.001, C: 0.001, Accuracy: 0.5673575129533679
linear, Gamma: 0.001, C: 0.01, Accuracy: 0.44892672094744634
linear, Gamma: 0.001, C: 0.1, Accuracy: 0.009622501850481125
linear, Gamma: 0.001, C: 1, Accuracy: 0.009622501850481125
linear, Gamma: 0.001, C: 10, Accuracy: 0.009622501850481125
linear, Gamma: 0.001, C: 100, Accuracy: 0.009622501850481125


In [ ]:
gkhjhk

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

# Define the SVM base classifier
svm_classifier = SVC()

# Create the bagging classifier with SVM as the base estimator
bagging_classifier = BaggingClassifier(estimator=svm_classifier, random_state=7)

# Use the best parameters to create the final bagging classifier
final_bagging_classifier = BaggingClassifier(estimator=SVC(C=10, gamma=10, kernel='rbf', random_state=7), 
                                             n_estimators=100,
                                             random_state=7)

# Train the final bagging classifier on the training data
final_bagging_classifier.fit(X_ss_train_sample, y_ss_train_sample)

# Evaluate the classifier on the validation set
val_predictions = final_bagging_classifier.predict(X_ss_val_sample)
val_accuracy = accuracy_score(y_ss_val_sample, val_predictions)
print("Validation Accuracy:", val_accuracy)



# Evaluate the classifier on the test set
#test_predictions = final_bagging_classifier.predict(X_test)
#test_accuracy = accuracy_score(y_test, test_predictions)
#print("Test Accuracy:", test_accuracy)


Validation Accuracy: 0.7838638045891931


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

# Define the SVM base classifier
svm_classifier = SVC()

# Create the bagging classifier with SVM as the base estimator
bagging_classifier = BaggingClassifier(estimator=svm_classifier, random_state=7)

# Use the best parameters to create the final bagging classifier
final_bagging_classifier = BaggingClassifier(estimator=SVC(C=10, gamma=10, kernel='rbf', random_state=7), 
                                             n_estimators=50,
                                             random_state=7)

# Train the final bagging classifier on the training data
final_bagging_classifier.fit(X_ss_train_sample, y_ss_train_sample)

# Evaluate the classifier on the validation set
val_predictions = final_bagging_classifier.predict(X_ss_val_sample)
val_accuracy = accuracy_score(y_ss_val_sample, val_predictions)
print("Validation Accuracy:", val_accuracy)



# Evaluate the classifier on the test set
#test_predictions = final_bagging_classifier.predict(X_test)
#test_accuracy = accuracy_score(y_test, test_predictions)
#print("Test Accuracy:", test_accuracy)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

# Define the SVM base classifier
svm_classifier = SVC()

# Define the parameters to tune (you can customize this based on your needs)
param_grid = {
    'base_estimator__C': [0.1, 1, 10],
    'base_estimator__kernel': ['linear', 'rbf', 'poly'],
    'base_estimator__gamma': ['scale', 'auto'],
    'n_estimators': [10, 50, 100],
}

# Create the bagging classifier with SVM as the base estimator
bagging_classifier = BaggingClassifier(base_estimator=svm_classifier, random_state=42)

# Create the GridSearchCV object for hyperparameter tuning
grid_search = GridSearchCV(bagging_classifier, param_grid, cv=3, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_ss_train_sample, y_ss_train_sample)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Use the best parameters to create the final bagging classifier
final_bagging_classifier = BaggingClassifier(base_estimator=SVC(**best_params['base_estimator']), 
                                             n_estimators=best_params['n_estimators'],
                                             random_state=42)

# Train the final bagging classifier on the training data
final_bagging_classifier.fit(X_ss_train_sample, y_ss_train_sample)

# Evaluate the classifier on the validation set
val_predictions = final_bagging_classifier.predict(X_ss_val_sample)
val_accuracy = accuracy_score(y_ss_val_sample, val_predictions)
print("Validation Accuracy:", val_accuracy)



# Evaluate the classifier on the test set
#test_predictions = final_bagging_classifier.predict(X_test)
#test_accuracy = accuracy_score(y_test, test_predictions)
#print("Test Accuracy:", test_accuracy)


C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\carth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-pac

https://datascience.stackexchange.com/questions/66216/gridsearch-without-cv

Autoencoder

In [12]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers

In [15]:
k_list = ['linear', 'rbf', 'sigmoid']
gamma_list = [0.001, 0.01, 0.1, 1, 10, 100,200, 300]
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 200, 300]

In [16]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the architecture of the autoencoder
input_dim = X_ss_train_sample.shape[1]
encoding_dim = 10  # You can adjust this based on your preference

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_ss_train_sample, X_ss_train_sample, epochs=50, batch_size=32, shuffle=True, validation_data=(X_ss_test_sample, X_ss_test_sample))

# Extract features using the encoder part of the autoencoder
encoder = Model(input_layer, encoded)
encoded_features_train = encoder.predict(X_ss_train_sample)
encoded_features_test = encoder.predict(X_ss_test_sample)

best_accuracy = 0
# Now use SVM as the classifier
for k in k_list:
    for c in c_list:
        for g in gamma_list:
            svm_classifier = SVC(C=c, gamma=g, kernel=k, random_state=7)  # You can adjust the kernel and C parameter based on your preference
            svm_classifier.fit(encoded_features_train, y_ss_train_sample)

            # Predict on the test set
            svm_predictions = svm_classifier.predict(encoded_features_test)

            # Evaluate accuracy
            accuracy = accuracy_score(y_ss_test_sample, svm_predictions)
            print(f'Kernel: {k}, C: {c}, Gamma: {g}, SVM Accuracy: {accuracy}')


            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

Epoch 1/50
451/451 [==============================] - 1s 1ms/step - loss: 0.9371 - val_loss: 0.7496
Epoch 2/50
451/451 [==============================] - 0s 975us/step - loss: 0.7085 - val_loss: 0.6479
Epoch 3/50
451/451 [==============================] - 0s 962us/step - loss: 0.6423 - val_loss: 0.6052
Epoch 4/50
451/451 [==============================] - 0s 992us/step - loss: 0.6095 - val_loss: 0.5841
Epoch 5/50
451/451 [==============================] - 0s 996us/step - loss: 0.5929 - val_loss: 0.5748
Epoch 6/50
451/451 [==============================] - 0s 994us/step - loss: 0.5855 - val_loss: 0.5706
Epoch 7/50
451/451 [==============================] - 0s 1ms/step - loss: 0.5821 - val_loss: 0.5682
Epoch 8/50
451/451 [==============================] - 0s 985us/step - loss: 0.5803 - val_loss: 0.5668
Epoch 9/50
451/451 [==============================] - 0s 1ms/step - loss: 0.5791 - val_loss: 0.5656
Epoch 10/50
451/451 [==============================] - 0s 970us/step - loss: 0.5781 - va

In [17]:
k_list = ['rbf']
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
c_list = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [19]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the architecture of the autoencoder
input_dim = X_ss_train_sample.shape[1]
encoding_dim = 10  # You can adjust this based on your preference

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_ss_train_sample, X_ss_train_sample, epochs=50, batch_size=32, shuffle=True, validation_data=(X_ss_test_sample, X_ss_test_sample))

# Extract features using the encoder part of the autoencoder
encoder = Model(input_layer, encoded)
encoded_features_train = encoder.predict(X_ss_train_sample)
encoded_features_test = encoder.predict(X_ss_test_sample)

best_accuracy = 0
# Now use SVM as the classifier
for k in k_list:
    for c in c_list:
        for g in gamma_list:
            svm_classifier = SVC(C=c, gamma=g, kernel=k, random_state=7)  # You can adjust the kernel and C parameter based on your preference
            svm_classifier.fit(encoded_features_train, y_ss_train_sample)

            # Predict on the test set
            svm_predictions = svm_classifier.predict(encoded_features_test)

            # Evaluate accuracy
            accuracy = accuracy_score(y_ss_test_sample, svm_predictions)
            print(f'Kernel: {k}, C: {c}, Gamma: {g}, SVM Accuracy: {accuracy}')


            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': g, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

Epoch 1/50
451/451 [==============================] - 1s 1ms/step - loss: 0.9399 - val_loss: 0.7243
Epoch 2/50
451/451 [==============================] - 0s 946us/step - loss: 0.6675 - val_loss: 0.6159
Epoch 3/50
451/451 [==============================] - 0s 977us/step - loss: 0.6126 - val_loss: 0.5879
Epoch 4/50
451/451 [==============================] - 0s 973us/step - loss: 0.5947 - val_loss: 0.5768
Epoch 5/50
451/451 [==============================] - 0s 998us/step - loss: 0.5870 - val_loss: 0.5715
Epoch 6/50
451/451 [==============================] - 0s 976us/step - loss: 0.5829 - val_loss: 0.5683
Epoch 7/50
451/451 [==============================] - 0s 976us/step - loss: 0.5804 - val_loss: 0.5663
Epoch 8/50
451/451 [==============================] - 0s 981us/step - loss: 0.5786 - val_loss: 0.5645
Epoch 9/50
451/451 [==============================] - 0s 961us/step - loss: 0.5771 - val_loss: 0.5629
Epoch 10/50
451/451 [==============================] - 0s 958us/step - loss: 0.5755 

In [9]:
k_list = ['linear', 'rbf', 'sigmoid']
gamma_list = [0.001, 0.01, 0.1, 1, 10, 100,200, 300]
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 200, 300]

In [12]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from keras import regularizers
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the architecture of the autoencoder
input_dim = X_ss_train_sample.shape[1]
encoding_dim = 10  # You can adjust this based on your preference

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_ss_train_sample, X_ss_train_sample, epochs=150, batch_size=32, shuffle=True, validation_data=(X_ss_test_sample, X_ss_test_sample))

# Extract features using the encoder part of the autoencoder
encoder = Model(input_layer, encoded)
encoded_features_train = encoder.predict(X_ss_train_sample)
encoded_features_test = encoder.predict(X_ss_test_sample)

best_accuracy = 0
# Now use SVM as the classifier
for k in k_list:
    for c in c_list:
        for g in gamma_list:
            svm_classifier = SVC(C=c, gamma=g, kernel=k, random_state=7)  # You can adjust the kernel and C parameter based on your preference
            svm_classifier.fit(encoded_features_train, y_ss_train_sample)

            # Predict on the test set
            svm_predictions = svm_classifier.predict(encoded_features_test)

            # Evaluate accuracy
            accuracy = accuracy_score(y_ss_test_sample, svm_predictions)
            print(f'Kernel: {k}, C: {c}, Gamma: {g}, SVM Accuracy: {accuracy}')


            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': g, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

Epoch 1/150
451/451 [==============================] - 1s 1ms/step - loss: 0.8731 - val_loss: 0.7044
Epoch 2/150
451/451 [==============================] - 0s 978us/step - loss: 0.6529 - val_loss: 0.6349
Epoch 3/150
451/451 [==============================] - 0s 977us/step - loss: 0.6110 - val_loss: 0.6113
Epoch 4/150
451/451 [==============================] - 0s 980us/step - loss: 0.5953 - val_loss: 0.6011
Epoch 5/150
451/451 [==============================] - 0s 1ms/step - loss: 0.5884 - val_loss: 0.5964
Epoch 6/150
451/451 [==============================] - 0s 995us/step - loss: 0.5849 - val_loss: 0.5938
Epoch 7/150
451/451 [==============================] - 0s 976us/step - loss: 0.5828 - val_loss: 0.5919
Epoch 8/150
451/451 [==============================] - 0s 969us/step - loss: 0.5813 - val_loss: 0.5904
Epoch 9/150
451/451 [==============================] - 0s 973us/step - loss: 0.5801 - val_loss: 0.5892
Epoch 10/150
451/451 [==============================] - 0s 974us/step - loss:

Autoencoder but with all 8 variables?
PCA instead of autoencoder?

In [9]:
k_list = ['rbf']
gamma_list = [0.001, 0.01, 0.1, 1, 10, 100,200, 300]
c_list = [0.001, 0.01, 0.1, 1, 10, 100, 200, 300]

In [17]:
best_accuracy = 0

# Import necessary libraries
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


for n in range(2,8):
# Perform PCA to reduce dimensionality
    #n_components = 2  # Number of principal components to keep
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_ss_train_sample)
    X_test_pca = pca.transform(X_ss_test_sample)

    for k in k_list:
        for c in c_list:
            for g in gamma_list:
                svm_classifier = SVC(C=c, gamma=g, kernel=k, random_state=7)  # You can adjust the kernel and C parameter based on your preference
                svm_classifier.fit(X_train_pca, y_ss_train_sample)

                y_pred = svm_classifier.predict(X_test_pca)

                accuracy = accuracy_score(y_ss_test_sample, y_pred)
                print(f'PCs: {n}, Kernel: {k}, C: {c}, Gamma: {g}, SVM Accuracy: {accuracy}')
                
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_parameters = {'PCs': n, 'C': c, 'gamma': g, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

PCs: 2, Kernel: rbf, C: 0.001, Gamma: 0.001, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 0.01, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 0.1, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 1, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 10, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 100, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 200, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.001, Gamma: 300, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.01, Gamma: 0.001, SVM Accuracy: 0.007401924500370096
PCs: 2, Kernel: rbf, C: 0.01, Gamma: 0.01, SVM Accuracy: 0.5692079940784603
PCs: 2, Kernel: rbf, C: 0.01, Gamma: 0.1, SVM Accuracy: 0.535899333826795
PCs: 2, Kernel: rbf, C: 0.01, Gamma: 1, SVM Accuracy: 0.41968911917098445
PCs: 2, Kernel: rbf, C: 0.01, Gamma: 10, SVM Accuracy: 0.2442635085122131

In [18]:
print("SS")
best_accuracy = 0

for k in k_list:
    for gamma in gamma_list:
        for c in c_list:
        # for each combination of parameters, train an SVC
                
            model = SVC(kernel=k, gamma=gamma, C=c, random_state = 7)
            model.fit(X_ss_train_sample, y_ss_train_sample)
            y_ss_pred = model.predict(X_ss_val_sample)
            accuracy = accuracy_score(y_ss_val_sample, y_ss_pred)
            print(f"{k}, Gamma: {gamma}, C: {c}, Accuracy: {accuracy}")

        # if we got a better score, store the score and parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = {'C': c, 'gamma': gamma, 'kernel': k}

print("Best score: {:.2f}".format(best_accuracy))
print("Best parameters: {}".format(best_parameters))

SS
rbf, Gamma: 0.001, C: 0.001, Accuracy: 0.009622501850481125
rbf, Gamma: 0.001, C: 0.01, Accuracy: 0.009622501850481125
rbf, Gamma: 0.001, C: 0.1, Accuracy: 0.5869726128793487
rbf, Gamma: 0.001, C: 1, Accuracy: 0.6036269430051814
rbf, Gamma: 0.001, C: 10, Accuracy: 0.5995558845299778
rbf, Gamma: 0.001, C: 100, Accuracy: 0.6262028127313102
rbf, Gamma: 0.001, C: 200, Accuracy: 0.6365655070318282
rbf, Gamma: 0.001, C: 300, Accuracy: 0.6406365655070319
rbf, Gamma: 0.01, C: 0.001, Accuracy: 0.009622501850481125
rbf, Gamma: 0.01, C: 0.01, Accuracy: 0.5840118430792006
rbf, Gamma: 0.01, C: 0.1, Accuracy: 0.6051073279052553
rbf, Gamma: 0.01, C: 1, Accuracy: 0.5888230940044411
rbf, Gamma: 0.01, C: 10, Accuracy: 0.6461880088823094
rbf, Gamma: 0.01, C: 100, Accuracy: 0.6724648408586232
rbf, Gamma: 0.01, C: 200, Accuracy: 0.6658031088082902
rbf, Gamma: 0.01, C: 300, Accuracy: 0.6650629163582531
rbf, Gamma: 0.1, C: 0.001, Accuracy: 0.0458919319022946
rbf, Gamma: 0.1, C: 0.01, Accuracy: 0.584381939

Used SS scaled data, best parameters as chosen above and all of the features

In [24]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = SVC(kernel="rbf", random_state=7, C=100, gamma=1)

#Train the model using the training sets
clf.fit(X_ss_train, y_ss_train)

#Predict the response for test dataset
y_pred = clf.predict(X_ss_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_ss_test, y_pred))

Accuracy: 0.9267633779883058
